# Đồ án cuối kỳ

(Cập nhật lần cuối: 12/09/2021)

Họ tên: Lê Hồng Quang

MSSV: 18127190

Họ tên: Lê Thanh Sơn

MSSV: 18127199

# 2. Khám phá dữ liệu

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
set_config(display='diagram')

### Đọc dữ liệu từ file

In [2]:
df = pd.read_csv('data.csv', sep =  ',', index_col = 0)

df.head()

,Ranking,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,1,China,"1,439,323,776",0.39 %,"5,540,090",153,"9,388,211","-348,399",1.7,38,61 %,18.47 %
1,2,India,"1,380,004,385",0.99 %,"13,586,631",464,"2,973,190","-532,687",2.2,28,35 %,17.70 %
2,3,United States,"331,002,651",0.59 %,"1,937,734",36,"9,147,420","954,806",1.8,38,83 %,4.25 %
3,4,Indonesia,"273,523,615",1.07 %,"2,898,047",151,"1,811,570","-98,955",2.3,30,56 %,3.51 %
4,5,Pakistan,"220,892,340",2.00 %,"4,327,022",287,"770,880","-233,379",3.6,23,35 %,2.83 %


#### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [3]:
df.shape

(235, 12)

#### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng trong DataFrame `df` cho biết dữ liệu liên quan đến dân số của các quốc gia và vùng lãnh thổ trên thế giới (cập nhật đến 2020). Có vẻ không có vấn đề các dòng có ý nghĩa khác nhau.

#### Dữ liệu có các dòng bị lặp không?

In [4]:
df.index.duplicated().sum()

0

#### Mỗi cột có ý nghĩa gì?

<b>VARIABLE DESCRIPTIONS:</b><br/>

- <b>Ranking</b>: Xếp hạng quốc gia theo dân số<br/>


- <b>Country (or dependency)</b>: Tên quốc gia (hoặc vùng lãnh thổ)<br/>


- <b>Population (2020)</b>: Dân số (2020)<br/>


- <b>Yearly Change</b>: Tỉ lệ thay đổi hằng năm<br/>


- <b>Net Change</b>: Dân số tăng lên hằng năm<br/>


- <b>Density (P/Km²)</b>: Mật độ dân số (Người/Km²)<br/>


- <b>Land Area (Km²)</b>: Diện tích (Km²)<br/>


- <b>Migrants (net)</b>:  Số dân di cư đến (người)<br/>


- <b>Fert. Rate</b>:  Tỉ lệ sinh sản (%)<br/>


- <b>Med. Age</b>:  Độ tuổi trung vị (độ tuổi chia dân số thành hai nhóm với số lượng bằng nhau)<br/>


- <b>Urban Pop %</b>:  Tỉ lệ dân thành thị (%)<br/>


- <b>World Share</b>: Tỉ lệ dân số so với toàn thế giới (%)<br/>

# 3. Đặt câu hỏi

Đặt câu hỏi tại đây

# 4. Tiền xử lý

### 4.1. Tiền xử lý kiểu dữ liệu

In [5]:
# Dataframe hiện đang có dữ liệu gì
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235 entries, 0 to 234
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Ranking                  235 non-null    int64 
 1   Country (or dependency)  235 non-null    object
 2   Population (2020)        235 non-null    object
 3   Yearly Change            235 non-null    object
 4   Net Change               235 non-null    object
 5   Density (P/Km²)          235 non-null    object
 6   Land Area (Km²)          235 non-null    object
 7   Migrants (net)           235 non-null    object
 8   Fert. Rate               235 non-null    object
 9   Med. Age                 235 non-null    object
 10  Urban Pop %              235 non-null    object
 11  World Share              235 non-null    object
dtypes: int64(1), object(11)
memory usage: 32.0+ KB


Khi đọc dữ liệu từ html và truyền vào df thì các dữ liệu đều được trả về kiểu `object` nên chúng ta phải thực hiện xử lý và trả về kiểu dữ liệu phù hợp (với ý nghĩa của cột).

#### 4.1.a. Kiểu dữ liệu số nguyên (int64)

Các cột có thể trả về kiểu dữ liệu số nguyên là **Population (2020)**, **Net Change**, **Migrants (net)**, **Med. Age** 

In [6]:
def removeComma(string):
    if string == string and string != ' ' and string != 'N.A.':
        return string.replace(",", "")
    return np.nan
df['Population (2020)'] = df['Population (2020)'].apply(removeComma)
df['Net Change'] = df['Net Change'].apply(removeComma)
df['Migrants (net)'] = df['Migrants (net)'].apply(removeComma)
df['Med. Age'] = df['Med. Age'].apply(removeComma)

df['Population (2020)'] = df['Population (2020)'].astype('Int64')
df['Net Change'] = df['Net Change'].astype('Int64')
df['Migrants (net)'] = df['Net Change'].astype('Int64')
df['Med. Age'] = df['Med. Age'].astype('Int64')

#### 4.1.b. Kiểu dữ liệu số thực (float64)

Các cột còn lại (ngoài cột tên Quốc gia - **Country (or dependency)**) đều có thể trả về dạng số thực. Tuy nhiên, có 3 cột **Yearly Change**, **Urban Pop %**, **World Share** đang ở dạng % vì vậy cần phải loại bỏ kí tự % trước khi ép kiểu dữ liệu.

In [7]:
df['Density (P/Km²)'] = df['Density (P/Km²)'].apply(removeComma)
df['Land Area (Km²)'] = df['Land Area (Km²)'].apply(removeComma)
df['Fert. Rate'] = df['Fert. Rate'].apply(removeComma)

df['Density (P/Km²)'] = df['Density (P/Km²)'].astype('float64')
df['Land Area (Km²)'] = df['Land Area (Km²)'].astype('float64')
df['Fert. Rate'] = df['Fert. Rate'].astype('float64')

In [8]:
def removePercent(string):
    string = removeComma(string)
    if string == string:
        return string.replace(" %", "")
    return np.nan

df['Yearly Change'] = df['Yearly Change'].apply(removePercent)
df['Urban Pop %'] = df['Urban Pop %'].apply(removePercent)
df['World Share'] = df['World Share'].apply(removePercent)

df['Yearly Change'] = df['Yearly Change'].astype('float64')
df['Urban Pop %'] = df['Urban Pop %'].apply('float64')
df['World Share'] = df['World Share'].apply('float64')

df = df.rename(columns={'Yearly Change': 'Yearly Change (%)', 'Urban Pop %': 'Urban Pop (%)', 'World Share': 'World Share (%)'})

In [9]:
df.head()

,Ranking,Country (or dependency),Population (2020),Yearly Change (%),Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop (%),World Share (%)
0,1,China,1439323776,0.39,5540090,153.0,9388211.0,5540090,1.7,38,61.0,18.47
1,2,India,1380004385,0.99,13586631,464.0,2973190.0,13586631,2.2,28,35.0,17.70
2,3,United States,331002651,0.59,1937734,36.0,9147420.0,1937734,1.8,38,83.0,4.25
3,4,Indonesia,273523615,1.07,2898047,151.0,1811570.0,2898047,2.3,30,56.0,3.51
4,5,Pakistan,220892340,2.00,4327022,287.0,770880.0,4327022,3.6,23,35.0,2.83


### 4.2. Tách các tập

### 4.3. Khám phá tập huấn luyện (train test)

### 4.4. Thay thế giá trị thiếu

# 5. Mô hình hóa

# 6. Nhìn lại quá trình làm đồ án

### 6.1 Những khó khăn đã gặp phải

TEXT

### 6.2 Những kỹ năng đã học được

TEXT

### 6.3 Nếu có thêm thời gian:

Nhóm sẽ tìm và thu thập dữ liệu đa dạng, phức tạp hơn, có thể thử nghiệm thu thập dữ liệu bằng cách get API trang web và mô hình hóa dữ liệu với nhiều thuật toán khác.

# 7. Tài liệu tham khảo

Chủ yếu từ các trang web:
- https://www.worldometers.info/world-population/population-by-country/<br/>
- https://scikit-learn.org/ <br/>
- https://stackoverflow.com/<br/>
- https://www.w3schools.com/<br/>

Video bài giảng + demo của môn học + slide bài giảng<br/>
Lab: 1, 2, 3